In [2]:
# made by Gibeom LEE, HI LAB
# 1M to 100k.ipynb 와 달리, folder_path안에 있는 모든 csv파일을 전부 축소 시켜줘서, 다른 폴더에 저장해주는 코드입니다.

In [3]:
import os
import glob
import numpy as np
import pandas as pd
from scipy.signal import find_peaks

In [4]:
# 원본 CSV 파일이 있는 폴더 경로
folder_path = r"C:\Users\GCU\OneDrive - gachon.ac.kr\HI Lab\0. Projects\0. On going\2. Aloe inspired DEG\0. 실험자료\3. Material selection\250802_flat 비교 및 dodecane 재평가\csv파일"

# 결과를 저장할 새 폴더 경로 생성
parent_dir = os.path.dirname(folder_path)
original_folder_name = os.path.basename(folder_path)
new_folder_name = f"minimized_{original_folder_name}"
output_folder = os.path.join(parent_dir, new_folder_name)
os.makedirs(output_folder, exist_ok=True)

# 모든 CSV 파일 경로 수집
csv_files = glob.glob(os.path.join(folder_path, "*.csv"))

# 파라미터 설정
height = 0.01  # 피크 감지 기준 전압 크기 (노이즈 레벨 고려)
num_samples = 10000 # 최종 줄이고자 하는 행 수

for file_path in csv_files:
    try:
        # CSV 파일 읽기 (헤더 없음)
        df = pd.read_csv(file_path, header=None, names=["Time", "Voltage"], skiprows=2)

        # 피크 검출
        peaks_positive, _ = find_peaks(df["Voltage"], height=height, distance=100)
        peaks_negative, _ = find_peaks(-df["Voltage"], height=height, distance=100)
        peaks = np.sort(np.concatenate((peaks_positive, peaks_negative)))

        # 피크 개수 확인
        num_peaks = len(peaks)
        print(f"{os.path.basename(file_path)} → Peak count: {num_peaks}")

        if num_peaks >= num_samples:
            selected_indices = np.sort(np.random.choice(peaks, num_samples, replace=False))
        else:
            num_non_peak_samples = num_samples - num_peaks
            non_peak_indices = np.setdiff1d(np.arange(len(df)), peaks)
            selected_non_peaks = np.linspace(0, len(non_peak_indices) - 1, num_non_peak_samples, dtype=int)
            selected_non_peaks = non_peak_indices[selected_non_peaks]

            selected_indices = np.sort(np.concatenate((peaks, selected_non_peaks)))

        # 축소된 데이터 생성
        filtered_df = df.iloc[selected_indices]

        # 새 파일 이름 및 저장 경로
        filename = os.path.basename(file_path)
        save_path = os.path.join(output_folder, filename)
        filtered_df.to_csv(save_path, index=False, header=False)

    except Exception as e:
        print(f"Error processing {file_path}: {e}")

A_Dodecane_2cm.csv → Peak count: 7549
A_Dodecane_2cm_12Hz.csv → Peak count: 7601
A_Dodecane_4cm.csv → Peak count: 7544
A_Dodecane_4cm_12Hz.csv → Peak count: 7593
A_Dodecane_4cm_16Hz_1min.csv → Peak count: 7580
A_Dodecane_4cm_16Hz_5min.csv → Peak count: 7196
A_Dodecane_6cm.csv → Peak count: 7585
A_Dodecane_6cm_12Hz.csv → Peak count: 7619
A_Dodecane_8cm.csv → Peak count: 7527
A_Dodecane_8cm_12Hz.csv → Peak count: 7604
A_lessPFPE_2cm.csv → Peak count: 7367
A_lessPFPE_4cm.csv → Peak count: 7271
A_lessPFPE_6cm.csv → Peak count: 7310
A_lessPFPE_8cm.csv → Peak count: 7499
A_muchPFPE_2cm.csv → Peak count: 7646
A_muchPFPE_4cm.csv → Peak count: 7539
A_muchPFPE_6cm.csv → Peak count: 7425
A_muchPFPE_8cm.csv → Peak count: 7524
A_PDMS_2cm.csv → Peak count: 7020
A_PDMS_4cm.csv → Peak count: 7388
A_PDMS_6cm.csv → Peak count: 6124
A_PDMS_8cm.csv → Peak count: 6107
F_lessPFPE_2cm.csv → Peak count: 7577
F_lessPFPE_4cm.csv → Peak count: 7263
F_lessPFPE_6cm.csv → Peak count: 7185
F_lessPFPE_8cm.csv → Peak 